## Part 1: Preprocessing

In [26]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [27]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [28]:
# Create y_df with the Attrition and Department columns

y_df = attrition_df[['Attrition', 'Department']]  # Targets


In [29]:
# Create a list of at least 10 column names to use as X data

feature_cat = ['Age', 'DistanceFromHome','HourlyRate','JobSatisfaction','NumCompaniesWorked','PercentSalaryHike','PerformanceRating','WorkLifeBalance','YearsAtCompany','YearsSinceLastPromotion','OverTime']  # Features

# Create X_df using your selected columns
X_df = attrition_df[feature_cat]

# Show the data types for X_df
X_df.dtypes


Age                         int64
DistanceFromHome            int64
HourlyRate                  int64
JobSatisfaction             int64
NumCompaniesWorked          int64
PercentSalaryHike           int64
PerformanceRating           int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
OverTime                   object
dtype: object

In [30]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=78)

In [31]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train['OverTime'].value_counts()

OverTime
No     799
Yes    303
Name: count, dtype: int64

In [32]:
X_train['OverTime'] = X_train['OverTime'].map({'Yes': 1, 'No': 0})
X_test['OverTime'] = X_test['OverTime'].map({'Yes': 1, 'No': 0})

X_train.head()


,Age,DistanceFromHome,HourlyRate,JobSatisfaction,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,OverTime
591,33,16,69,1,5,15,3,3,3,0,0
267,25,5,85,1,1,12,3,3,6,1,0
1236,36,13,96,1,5,13,3,3,2,2,1
788,28,10,59,3,3,13,3,4,8,1,0
1224,26,17,62,3,1,15,3,4,3,0,0


In [33]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data

X_scaler = scaler.fit(X_train)
# Scale the training and testing data

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [34]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
enc = OneHotEncoder(sparse_output=False)


# Fit the encoder to the training data

encode_dept = enc.fit(y_train[['Department']])


# Create two new variables by applying the encoder
# to the training and testing data

y_train_enc_dept = enc.transform(y_train[['Department']])
y_test_enc_dept = enc.transform(y_test[['Department']])

y_train_enc_dept
y_test_enc_dept


array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [35]:
# Create a OneHotEncoder for the Attrition column
enc = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
encode_att = enc.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_enc_att = enc.transform(y_train[['Attrition']])
y_test_enc_att = enc.transform(y_test[['Attrition']])


## Create, Compile, and Train the Model

In [36]:
# Find the number of columns in the X training data
number_input_features = len(X_train_scaled[0])

# Create the input layer
input_layer = layers.Input(shape=(number_input_features,))

# Create at least two shared layers
shared1 = layers.Dense(64, activation='relu')(input_layer)
shared2 = layers.Dense(128, activation='relu')(shared1)


In [37]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_hidden = layers.Dense(32, activation='softmax', name='dept_layer')(shared2)

# Create the output layer

dept_output = layers.Dense(3, activation='softmax', name='dept_output')(dept_hidden)



In [38]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
att_hidden = layers.Dense(32, activation='relu', name='att_layer')(shared2)


# Create the output layer
att_output = layers.Dense(2, activation='sigmoid', name='att_output')(att_hidden)


In [39]:
# Create the model
model = Model(inputs=input_layer, outputs=[att_output, dept_output])


# Compile the model
model.compile(optimizer='adam',
              loss={'att_output': 'binary_crossentropy', 'dept_output': 'categorical_crossentropy'},
              metrics={'att_output': 'accuracy', 'dept_output': 'accuracy'})

# Summarize the model
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │        768 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │      8,320 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ att_layer (Dense)   │ (None, 32)        │      4,128 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dept_layer (Dense)  │ (None, 32)        │      4,128 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ att_output (Dense)  │ (None, 2)         │         66 │ att_layer[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dept_output (Dense) │ (None, 3)         │         99 │ dept_layer[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,509 (68.39 KB)

 Trainable params: 17,509 (68.39 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
# Train the model
model.fit(
    X_train_scaled,
    {'att_output': y_train_enc_att, 'dept_output': y_train_enc_dept},
    epochs=100,
    batch_size=35,
    validation_split=0.2
)


Epoch 1/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - att_output_accuracy: 0.7740 - att_output_loss: 0.5780 - dept_output_accuracy: 0.5170 - dept_output_loss: 1.0742 - loss: 1.6519 - val_att_output_accuracy: 0.8235 - val_att_output_loss: 0.4521 - val_dept_output_accuracy: 0.6606 - val_dept_output_loss: 1.0246 - val_loss: 1.4899
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - att_output_accuracy: 0.8295 - att_output_loss: 0.4406 - dept_output_accuracy: 0.6552 - dept_output_loss: 1.0092 - loss: 1.4503 - val_att_output_accuracy: 0.8235 - val_att_output_loss: 0.4273 - val_dept_output_accuracy: 0.6606 - val_dept_output_loss: 0.9688 - val_loss: 1.4056
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - att_output_accuracy: 0.8458 - att_output_loss: 0.4009 - dept_output_accuracy: 0.6907 - dept_output_loss: 0.9401 - loss: 1.3410 - val_att_output_accuracy: 0.8235 - val_att_output_loss: 0.4082 - val_dept_output_accuracy: 0.6606 - val_dept_output_loss: 0.9170 - val_loss: 1.3344
Epoch 4/100


In [41]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, {'att_output': y_test_enc_att, 'dept_output': y_test_enc_dept})
test_results


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - att_output_accuracy: 0.8407 - att_output_loss: 1.0468 - dept_output_accuracy: 0.5776 - dept_output_loss: 1.0112 - loss: 2.0608 


[2.0276010036468506,
 1.036394715309143,
 0.9726681709289551,
 0.845108687877655,
 0.595108687877655]

In [42]:
# Print the accuracy for both department and attrition
#print(f'Department predictions accuracy: {dept_output}')
accuracy_dept = test_results[4]
print(f"Department prediction accuracy {accuracy_dept}")

accuracy_att = test_results[3]
print(f"Attrition prediction accuracy {accuracy_att}")

Department prediction accuracy 0.595108687877655
Attrition prediction accuracy 0.845108687877655


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.  Accuracy may not be the best metric on this data as both the attrition and department targets are unbalanced.  Since attrition is binary, AUC-ROC may be a better measure.  Precision could also be used as a measurement for attrition since we are trying to predict if employees will leave the company.  For understanding if employees may be better suited for a different department recall may be a better measurement as it measures the ability of the model to find the relevant cases (true positives). 
2.  For the attrition output layer, the 'sigmoid' activation function was used beacause the attrition target was binary and I wanted a binary out, which would be achieved through the 'sigmoid' function.  'Softmax' activation function was used for department because there were three targets associated with department and the output was mutually exclusive - making 'softmax' the optimal activiation function for department. 
3.  The model might be improved by adding more layers, more neurons, or more epochs.  Additionally, adding more data (if possible) could be helpful in improving model performance.